<br>

## Loading libraries

In [ ]:
#core
import gym
import wandb
import numpy as np

# graphics
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

# sklearn
from sklearn.cluster import KMeans

wandb.login();

<br>
<br>
<br>

## Set up some helper functions

In [ ]:
def moving_average(data_set, periods=3):
    weights = np.ones(periods) / periods
    ma = np.convolve(data_set, weights, mode='valid')
    return np.append(np.repeat(np.nan, periods), ma)

def design_matrix(PARAMS):
    ls = np.linspace(env.observation_space.low, env.observation_space.high, PARAMS['n_states']).T
    X = np.array(np.meshgrid(ls[0], ls[1], range(env.action_space.n))).T.reshape(-1,3).round(3)
    actions = X[:,2].astype(int)
    onehot_actions = np.eye(env.action_space.n)[actions]
    X = np.concatenate((X[:,:2], onehot_actions), axis=1)
    x, y = np.meshgrid(ls[0], ls[1])
    return X, x, y

def get_value(state_act, rbf_settings):
    # Predicting the value of a given state-action combination
    w, kmeans, sig, J = rbf_settings
    xx = np.array([state_act])
    uu = np.exp(-np.linalg.norm(xx - kmeans.cluster_centers_, axis=1)/sig).reshape(1,-1)
    return np.dot(uu, w)[0] # [0]

def loss_surface(x, y, z, title, ax):
    
    ax.plot_surface(x, y, z, rstride=1, cstride=1, cmap='viridis', vmin=np.nanmin(z), vmax=np.nanmax(z), linewidth=0.08, antialiased=True)

    # make the panes transparent
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    
    # make the grid lines dashed
    ax.xaxis._axinfo['grid']['linestyle'] =  '--'
    ax.yaxis._axinfo['grid']['linestyle'] =  '--'
    ax.zaxis._axinfo['grid']['linestyle'] =  '--'

    ax.set_xlabel('Position', fontsize=12, labelpad=7)
    ax.set_ylabel('Velocity', fontsize=12, labelpad=10)
    ax.set_zlabel('Value', fontsize=12, labelpad=7)

    ax.set_title(title, fontsize=14)

In [ ]:
%%capture
env = gym.make('MountainCar-v0').unwrapped
wandb.init(project='MountainCarProblem')

In [ ]:
wandb.log({'mses_png': wandb.Image('./Plots/mses.png')})

<br>
<br>
<br>

## Plot steps taken per episode

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

ma = 30
tabular = wandb.Api().run(f'/MountainCarProblem/12rh1pdt').history(samples=5000)
regression75 = wandb.Api().run(f'/MountainCarProblem/2u60rotl').history(samples=5000)
regression450 = wandb.Api().run(f'/MountainCarProblem/1vbifh3g').history(samples=5000)

def left(ax=ax):
    
    ax.plot(moving_average(regression75['steps'], ma), label='RBF(75)', linewidth=1)
    ax.plot(moving_average(regression450['steps'], ma), label='RBF(450)', linewidth=1)
    ax.plot(moving_average(tabular['steps'], ma), label='Tabular', linewidth=1)
    ax.set_ylim([0, 2000])
    ax.grid()
    ax.legend()
    ax.set(xlabel='Episode', ylabel='Steps')
    
left()
sns.despine()

plt.savefig('./Plots/tabular_rbf.png', bbox_inches='tight', pad_inches=0.1)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

ma = 30
tabular = wandb.Api().run(f'/MountainCarProblem/12rh1pdt').history(samples=5000)
regression75 = wandb.Api().run(f'/MountainCarProblem/2u60rotl').history(samples=5000)
regression450 = wandb.Api().run(f'/MountainCarProblem/1vbifh3g').history(samples=5000)

def left(ax=ax[0]):
    
    ax.plot(moving_average(regression75['steps'], ma), label='RBF(75)', linewidth=1)
    ax.plot(moving_average(regression450['steps'], ma), label='RBF(450)', linewidth=1)
    ax.plot(moving_average(tabular['steps'], ma), label='Tabular', linewidth=1)
    ax.set_ylim([0, 2000])
    ax.grid()
    ax.legend()
    ax.set(title=f'Steps till completion, MA({ma})', xlabel='Episode', ylabel='Steps')
    
def right(ax=ax[1]):
    
    ax.plot(tabular['lr'], label='L-rates')
    ax.grid()
    ax.set(title='Learning rate schedule', xlabel='Episode', ylabel='Learning rate')
    
    
left()
right()
sns.despine()

# plt.savefig('./Plots/tabular_rbf.png', bbox_inches='tight', pad_inches=0.1)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

ma = 30
sarsa = wandb.Api().run(f'/MountainCarProblem/29tsllke').history(samples=5000)
qlearn = wandb.Api().run(f'/MountainCarProblem/1o61c32z').history(samples=5000)

def left(ax=ax[0]):
    
    ax.plot(moving_average(sarsa['steps'], ma), label='SARSA', linewidth=1)
    ax.plot(moving_average(qlearn['steps'], ma), label='Q-Learning', linewidth=1)
    ax.plot(moving_average(regression450['steps'], ma), label='RBF(450) offline', linewidth=1)
    ax.set_ylim([0, 2000])
    ax.grid()
    ax.legend()
    ax.set(title=f'Steps till completion, MA({ma})', xlabel='Episode', ylabel='Steps')
    
def right(ax=ax[1]):
    
    ax.plot(sarsa['lr'], label='L-rates')
    ax.grid()
    ax.set(title='Learning rate schedule', xlabel='Episode', ylabel='Learning rate')
    
       
left()
right()
sns.despine()

# plt.savefig('./Plots/<>.png', bbox_inches='tight', pad_inches=0.1)

<br>
<br>
<br>

## Create the loss surface plots

In [ ]:
# download the q_table for the tabular method
run = wandb.Api().run(f'/MountainCarProblem/12rh1pdt')
run.file('q_table.npy').download(replace=True)
q_table = np.load('./q_table.npy')
PARAMS = run.config

# transforming the data for a surface plot
X, x1, y1 = design_matrix(PARAMS)
z1 = q_table.max(axis=2)
z1[np.where(z1==0)] = np.nan

# plotting the loss surface
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection='3d', elev=60, azim=-60)
loss_surface(x1, y1, z1, 'Tabular method', ax)

plt.savefig('./Plots/tabular_loss.png', bbox_inches='tight', pad_inches=0.1)
wandb.log({'tabular_loss_png': wandb.Image(fig)})

In [ ]:
# loading the data from the run
run = wandb.Api().run(f'/MountainCarProblem/1vbifh3g')
run.file('weights.npy').download(replace=True)
weights = np.load('./weights.npy')
PARAMS = run.config

# transforming the data for a surface plot
X, x2, y2 = design_matrix(PARAMS)
kmeans = KMeans(n_clusters=PARAMS['J'], random_state=PARAMS['seed']).fit(X)
sig = np.std(X)
rbf_settings = (weights, kmeans, sig, PARAMS['J'])
z2 = np.array(list(map(lambda x: get_value(x, rbf_settings), X)))    
z2 = np.array(z2).reshape(env.action_space.n, PARAMS['n_states'], PARAMS['n_states']).max(axis=0)
# Treating extreme values
z2[z2>20] = 20
z2[z2<-100] = -100

# plotting the loss surface
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection='3d', elev=60, azim=-60)
loss_surface(x2, y2, z2, f'Offline RBF({PARAMS["J"]}) approximation', ax)

plt.savefig('./Plots/rbf_loss.png', bbox_inches='tight', pad_inches=0.1)
wandb.log({'rbf_loss_png': wandb.Image(fig)})

In [ ]:
# loading the data from the run
run = wandb.Api().run(f'/MountainCarProblem/29tsllke')
run.file('weights.npy').download(replace=True)
weights = np.load('./weights.npy')
PARAMS = run.config

# transforming the data for a surface plot
X, x3, y3 = design_matrix(PARAMS)
kmeans = KMeans(n_clusters=PARAMS['J'], random_state=PARAMS['seed']).fit(X)
sig = np.std(X)
rbf_settings = (weights, kmeans, sig, PARAMS['J'])
z3 = np.array(list(map(lambda x: get_value(x, rbf_settings), X)))    
z3 = np.array(z3).reshape(env.action_space.n, PARAMS['n_states'], PARAMS['n_states']).max(axis=0)
# Treating extreme values
z3[z3>20] = 20
z3[z3<-100] = -100

# plotting the loss surface
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection='3d', elev=30, azim=-60)
loss_surface(x3, y3, z3, f'SARSA', ax)

plt.savefig('./Plots/sarsa_loss.png', bbox_inches='tight', pad_inches=0.1)
wandb.log({'sarsa_loss_png': wandb.Image(fig)})

In [ ]:
# loading the data from the run
run = wandb.Api().run(f'/MountainCarProblem/1o61c32z')
run.file('weights.npy').download(replace=True)
weights = np.load('./weights.npy')
PARAMS = run.config

# transforming the data for a surface plot
X, x4, y4 = design_matrix(PARAMS)
kmeans = KMeans(n_clusters=PARAMS['J'], random_state=PARAMS['seed']).fit(X)
sig = np.std(X)
rbf_settings = (weights, kmeans, sig, PARAMS['J'])
z4 = np.array(list(map(lambda x: get_value(x, rbf_settings), X)))    
z4 = np.array(z4).reshape(env.action_space.n, PARAMS['n_states'], PARAMS['n_states']).max(axis=0)
# Treating extreme values
z4[z4>20] = 20
z4[z4<-100] = -100

# plotting the loss surface
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection='3d', elev=30, azim=-60)
loss_surface(x4, y4, z4, f'Q-Learning', ax)

plt.savefig('./Plots/qlearn_loss.png', bbox_inches='tight', pad_inches=0.1)
wandb.log({'qlearn_loss_png': wandb.Image(fig)})

In [ ]:
fig = plt.figure(figsize=(14, 6))
spec = gridspec.GridSpec(ncols=3, nrows=1)
ax3d1 = fig.add_subplot(spec[0], projection='3d', elev=60, azim=-60)
ax3d2 = fig.add_subplot(spec[1], projection='3d', elev=60, azim=-60)
ax3d3 = fig.add_subplot(spec[2], projection='3d', elev=30, azim=-60)

loss_surface(x1, y1, z1, 'Tabular method', ax3d1)
loss_surface(x2, y2, z2, 'Offline RBF(450) approximation', ax3d2)
loss_surface(x4, y4, z4, 'Q-Learning', ax3d3)

plt.savefig('./Plots/loss_surfaces.png', bbox_inches='tight', pad_inches=0.1)